In [1]:
import requests, pymysql, math, re, time, csv
from bs4 import BeautifulSoup

In [2]:
## 회사 코드 csv불러오기 ##

companycode=list()
with open("companycode.csv",'r') as f:
    reader = csv.reader(f)
    
    for code in reader:
        companycode.append(code[0])
total = len(companycode)

In [3]:
## 회사 정보 크롤링 함수 ##

def crawl(code):
    url = "https://thevc.kr/"
    html = ""
    req = requests.get(url+code)
    if req.status_code == 200 :
        html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    
    brand = []    
    brd_code = code
    brd_link = str()
    service_link = str()
    service_name = str()
    service_info = str()
    service_detail = str()
    
    for info in soup.find_all("div", class_="contact_info info_box"):
        for span in info.find_all("span"):
            href = span.find("a").get("href")
            if href[0:4] == "http":
                brd_link = href
    
    for prdt in soup.find_all("div", class_="info_box product_box"):
        name=prdt.find("span", class_="product_name")
        if name!=None:
            service_name=name.text
        else:
            service_name=None
            
        link = prdt.find("a", class_="product_link")
        if link!=None:
            service_link=link.get("href")
        else:
            service_link=None
            
        detail=prdt.find("p")
        if detail!=None:
            detail=str(detail).replace('<p>','').replace('</p>','').replace('<span>','{').replace('</span>','}').strip()
            service_detail=detail
        else:
            service_detail=None

        for span in prdt.select("div > span"):
            class_=span.get("class")
            if class_==None:
                service_info=span.text
            else:
                service_info=None
        
        brand.append([brd_code, brd_link, service_link, service_name, service_info, service_detail])
    
    return brand

In [4]:
## 회사 정보 배열 ##

comp_info = list()
cnt=0

In [13]:
## 크롤링 함수 실행 ##
startTime = time.time()

for comp in companycode[1400:1500]:
    craw_comp = crawl(comp)
    comp_info+=craw_comp
    cnt+=1
    if cnt%10==0:
        print(total, "中", cnt, '개 완료')

print("--- {0:.2f} seconds ---".format(time.time() - startTime))

2007 中 420 개 완료
2007 中 430 개 완료
2007 中 440 개 완료
2007 中 450 개 완료
2007 中 460 개 완료
2007 中 470 개 완료
2007 中 480 개 완료
2007 中 490 개 완료
2007 中 500 개 완료
2007 中 510 개 완료
--- 5.45 seconds ---


In [14]:
print(len(comp_info))
# comp_info

795


In [15]:
## 중간 저장 (985까지 완료) ##

comp_info2csv = open('company-infomation_987-1400.csv', 'w', newline='', encoding='UTF8')

csvwriter = csv.writer(comp_info2csv)
for row in comp_info:
    csvwriter.writerow(row)

comp_info2csv.close()